### Import modules

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers
import boto3
import json

### Initialize and configure OpenSearch client

In [ ]:
host = "HOST.us-west-2.aoss.amazonaws.com" #replace this with the AOSS endpoint
region = "us-west-2"
service = "aoss"
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

In [ ]:
client = OpenSearch(
    hosts = [{"host": host, "port": 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

### Initialize and configure Boto Client for Bedrock

In [ ]:
bedrock = boto3.client(
 service_name='bedrock',
 region_name='us-west-2',
 endpoint_url='https://bedrock.us-west-2.amazonaws.com'
)

### Define function for generating embeddings

In [ ]:
def text_embedding(text):
    body=json.dumps({"inputText": text})
    response = bedrock.invoke_model(body=body, modelId='amazon.titan-e1t-medium', accept='application/json', contentType='application/json')
    response_body = json.loads(response.get('body').read())
    embedding = response_body.get('embedding')
    return embedding

### Create the prompt

In [ ]:
prompt='Who won the Best Actor award in a supporting role?'

### Build context

In [ ]:
def search_index(vector):
    document = {
        "size": 15,
        "_source": {"excludes": ["nominee_vector"]},
        "query": {
            "knn": {
                 "nominee_vector": {
                     "vector": vector,
                     "k":15
                 }
            }
        }
    }
    response = client.search(
    body = document,
    index = "oscars-index"
    )
    return response

In [ ]:
vector=text_embedding(prompt)

In [ ]:
response=search_index(vector)
data=response['hits']['hits']

In [ ]:
data

In [ ]:
context = ''
for item in data:
    context += item['_source']['nominee_text'] + '\n'
print(context)

In [ ]:
augmented_prompt=f'Context - {context}\nBased on the above context, answer this question - {prompt}'

In [ ]:
augmented_prompt

### Invoke Titan LLM Endpoint

In [ ]:
config={
      "maxTokenCount": 1000,
      "stopSequences": [],
      "temperature":0.1,
      "topP":1
}

body = json.dumps({'inputText': augmented_prompt,'textGenerationConfig':config})

In [ ]:
response = bedrock.invoke_model( 
 modelId='amazon.titan-tg1-large', 
 body=body
)

In [ ]:
response_body = json.loads(response.get('body').read())
print(response_body.get('results')[0].get('outputText'))